## POS
I chose Russian(rus) as the source language to project the POS to Kazakh (kk) 

### Preprocessing the data
I spent a lot time to make the fastalign to work, and then I spent a lot time to find that udpipe model tokenize the original file into weird sentences in conllu format because there are many numbers as separate lines in the original files (ru.s or kk.s) and some sentences starts with a number which udpipe tokenize the number and the rest part of the sentence into two sentences in *conllu*, anyways I make the parallel sentence pairs first using your command line and then filtered all the lines with any numbers or has no paired parallel sentence as following. And I use this ***ru-kk_filtered*** parallel file to extract the source and target sentences again. In the end it results in 43550 sentences for both target and source languages.

In [60]:
import re

### get rid of empty alignments
lines = []
with open("./ru-kk", "r", encoding="utf-8") as f:
    for line in f:
        if re.match("([0-9]* \.)|([0-9]* , [0-9]* \.)", line):
            #print(line)
            continue
        if not line.strip():
            continue
        lines.append(line.strip())
        #print(line.strip())
with open("./ru-kk_filtered", "w", encoding="utf-8") as new_f:  
    for line in lines:
        print(line, file=new_f)

In [ ]:
with open("./ru-kk", "r", encoding="utf-8") as f:
    r_lines, k_lines = [], []
    for line in f:
        line = line.strip().split(" ||| ")
        r, k = line[0], line[1]
        r_lines.append(r)
        k_lines.append(k)

with open("./ru_filtered.s", "w", encoding="utf-8") as r_f:
    for line in r_lines:
        print(line, end="\n\n",file=r_f)
with open("./kk_filtered.s", "w", encoding="utf-8") as k_f:
    for line in k_lines:
        print(line, end="\n\n", file=k_f)

### Get conllu file from filtered source and target language files
- Source language (ru): I downloaded udpipe model of russian and run tokenize and tag on the ru_filtered.s file 
- Target language (kk): I don't understand how we should tokenize the target language to which there is supposed to have no trained tokenizer or tagger or parser due to its low resource. I in the end tokenized it using the russian udpipe model, given that we are going to align their sentences word by word later, it's a not bad idea to use the same tokenizer to tokenize them.

### Get alignment file
Use the ***ru-kk_filtered*** file to run with the fastalign forward and backward(reversed) and finally get the intersection of the both alignments, saved as ***ru-kk_filtered.i***

### Using the alignment file and the two conllu files and target language file kk_filtered.s to project Russian POS onto Kazakh sentences. 
I only projected every target word that is not aligned with POS "NOUN" because I ran out of time. The projected Kazakh conllu file with POS is saved as ***kk_projected_from_ru.conllu***.

### At last, I trained a udpipe model using ***kk_projected_from_ru.conllu*** file which is saved as **kk.udpipe** 
and use the test set downloaded from UD to test the accuracy which is expected to be very low: <br>

```

Tagging from gold tokenization - forms: 10007, upostag: 30.91%, xpostag: 0.00%, feats: 35.73%, alltags: 0.00%, lemmas: 0.00%

```